In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import datetime

# Paths to the uploaded CSV files
file_paths = [[f'data/Methane/M{j}_{i}.csv' for i in range(3)] for j in range(1,6)]

# Read all CSVs and store in a list
arrays = []
for paths in file_paths:
    array = []
    for path in paths:
        df = pd.read_csv(path)  # Set header=None if there's no header row
        array.append(df.values)
    array = np.vstack(array)
    arrays.append(array)

# Paths to the uploaded CSV files
file_paths = [[f'data/Ethylene/E{j}_{i}.csv' for i in range(3)] for j in range(1,2)]

# Read all CSVs and store in a list
for paths in file_paths:
    array = []
    for path in paths:
        df = pd.read_csv(path)  # Set header=None if there's no header row
        array.append(df.values)
    array = np.vstack(array)
    arrays.append(array)

In [ ]:
len(arrays)

In [ ]:
sensor_id = ["U1-TGS2602", "U2-TGS2610-D00", "U3-SP3S-AQ2", "U4-GSBT11-DXX",
             "U8-TGS2600", "U9-TGS2603", "U10-TGS2630", "U13-TGS2612-D00",
             "U14-TGS2620", "U15-MG-812", "U16-TGS-3830", "U19-TGS1820",
             "U20-TGS2611-E00", "U21-TGS2616-C00", "U22-WSP2110", "U25-TGS-3870",
             "U7-BME680"]

sensor_pos = [[0,1], [0,3], [2,2], [2,3],
              [0,0], [0,2], [2,0], [1,1],
              [1,3], [3,1], [3,3], [3,2],
              [1,0], [1,2], [3,0], [2,1]]

sensor_addr = [[0x49, 1],[0x49, 3],[0x4b, 2],[0x4b, 3],
               [0x49, 0],[0x49, 2],[0x4b, 0],[0x48, 1],
               [0x48, 3],[0x4a, 1],[0x4a, 3],[0x4a, 2],
               [0x48, 0],[0x48, 2],[0x4a, 0],[0x4b, 1],
               [0x76, 99]]

sensor_addr_to_tag = [[0x49, 0], [0x4b, 1], [0x48, 2], [0x4a, 3],]

adc_addr = [0x48, 0x49, 0x4a, 0x4b]

In [ ]:
fig = plt.figure(figsize=(10,15))
ax = fig.add_subplot(111)

fmt = '%Y-%m-%d %H:%M:%S.%f'

stack_data = []
stack_times = []

for array in arrays[:5]:
    time0 = datetime.strptime(array[0, 0], fmt)
    times = [(datetime.strptime(row[0], fmt) - time0).total_seconds() for row in array]
    sensor_data = []

    for i in range(16):
        values = []
        for read in array:
            values.append(int.from_bytes(read[2*i+2:2*i+4].tolist(), byteorder="little"))    # Check with Herman
        sensor_data.append(values)
    stack_data.append(np.vstack(sensor_data))
    stack_times.append(times)

    for i, row in enumerate(sensor_data):
        if i==0:
            baseline = 0.
            signal = (np.array(row) - baseline)/65536.
            ax.plot(times, signal, label=sensor_id[i], color='gray', linewidth=1, alpha=0.5)
        else:
            baseline = np.mean(row[:5])
            signal = (np.array(row) - baseline)/65536.
            ax.plot(times, signal*10 + 2*i , label=sensor_id[i+1], color='gray', linewidth=1, alpha=0.5)
    ax.set_yticks(2*np.arange(16), labels=sensor_id[:-1])
    ax.set_xticks([times[0], times[-1]], labels=[0,250])
    ax.set_xlabel('Time (s)')
    ax.spines[['top', 'right', 'left']].set_visible(False)
    ax.tick_params(left=False, labelleft=True)
    ax.set_xlim([times[0], times[-1]])
    ax.vlines(9, -1, 31, colors='black', linestyles='dashed', linewidth=0.5)
    ax.vlines(130, -1, 31, colors='black', linestyles='dashed', linewidth=0.5)
plt.savefig('figs/methane.pdf', bbox_inches='tight')
plt.savefig('figs/methane.png', dpi=200, bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(10,15))
ax = fig.add_subplot(111)

fmt = '%Y-%m-%d %H:%M:%S.%f'

stack_data = []
stack_times = []

for array in arrays[5:]:
    time0 = datetime.strptime(array[0, 0], fmt)
    times = [(datetime.strptime(row[0], fmt) - time0).total_seconds() for row in array]
    sensor_data = []

    for i in range(16):
        values = []
        for read in array:
            values.append(int.from_bytes(read[2*i+2:2*i+4].tolist(), byteorder="little"))    # Check with Herman
        sensor_data.append(values)
    stack_data.append(np.vstack(sensor_data))
    stack_times.append(times)

    for i, row in enumerate(sensor_data):
        if i==0:
            baseline = 0.
            signal = (np.array(row) - baseline)/65536.
            ax.plot(times, signal, label=sensor_id[i], color='gray', linewidth=1, alpha=0.5)
        else:
            baseline = np.mean(row[-5:])
            signal = (np.array(row) - baseline)/65536.
            ax.plot(times, signal*5 + 2*i , label=sensor_id[i+1], color='gray', linewidth=1, alpha=0.5)
    ax.set_yticks(2*np.arange(16), labels=sensor_id[:-1])
    ax.set_xticks([times[0], times[-1]], labels=[0,250])
    ax.set_xlabel('Time (s)')
    ax.spines[['top', 'right', 'left']].set_visible(False)
    ax.tick_params(left=False, labelleft=True)
    ax.set_xlim([times[0], times[-1]])
    ax.vlines(9, -1, 31, colors='black', linestyles='dashed', linewidth=0.5)
    ax.vlines(130, -1, 31, colors='black', linestyles='dashed', linewidth=0.5)

In [ ]:
fig = plt.figure(figsize=(10,15))
ax = fig.add_subplot(111)

fmt = '%Y-%m-%d %H:%M:%S.%f'

stack_data = []
stack_times = []

for array in arrays[:5]:
    time0 = datetime.strptime(array[0, 0], fmt)
    times = [(datetime.strptime(row[0], fmt) - time0).total_seconds() for row in array]
    sensor_data = []

    for i in range(16):
        values = []
        for read in array:
            values.append(int.from_bytes(read[2*i+2:2*i+4].tolist(), byteorder="little"))    # Check with Herman
        sensor_data.append(values)
    stack_data.append(np.vstack(sensor_data))
    stack_times.append(times)

    for i, row in enumerate(sensor_data):
        if i==0:
            baseline = 0.
            signal = (np.array(row) - baseline)/65536.
            ax.plot(times, signal, label=sensor_id[i], color='gray', linewidth=1, alpha=0.5)
        else:
            baseline = np.mean(row[:5])
            signal = (np.array(row) - baseline)/65536.
            ax.plot(times, signal*10 + 2*i , label=sensor_id[i+1], color='gray', linewidth=1, alpha=0.5)
    ax.set_yticks(2*np.arange(16), labels=sensor_id[:-1])
    ax.set_xticks([times[0], times[-1]], labels=[0,180])
    ax.set_xlabel('Time (s)')
    ax.spines[['top', 'right', 'left']].set_visible(False)
    ax.tick_params(left=False, labelleft=True)
    ax.set_xlim([times[0], times[-1]])
    ax.vlines(9, -1, 31, colors='black', linestyles='dashed', linewidth=0.5)
    ax.vlines(130, -1, 31, colors='black', linestyles='dashed', linewidth=0.5)

In [ ]:
# Data for ethylene
for array in arrays[5:]:
    time0 = datetime.strptime(array[0, 0], fmt)
    times = [(datetime.strptime(row[0], fmt) - time0).total_seconds() for row in array]
    sensor_data = []

    for i in range(16):
        values = []
        for read in array:
            values.append(int.from_bytes(read[2*i+2:2*i+4].tolist(), byteorder="little"))    # Check with Herman
        sensor_data.append(values)
    stack_data.append(np.vstack(sensor_data))
    stack_times.append(times)

In [ ]:
stack_response = []
responses = []
for data in stack_data[:5]:
    baseline = np.mean(data[:, 4:8], axis=1)
    onset_steady = np.mean(data[:, 110:125], axis=1)
    response = np.absolute(onset_steady - baseline)
    stack_response.append(response)
response = np.mean(np.vstack(stack_response), axis=0)
response /= np.max(response) - np.min(response)
response -= np.min(response)
# response = 2*response - 1
responses.append(response)

for data in stack_data[5:]:
    baseline = np.mean(data[:, 4:8], axis=1)
    onset_steady = np.mean(data[:, 110:125], axis=1)
    response = np.absolute(onset_steady - baseline)
    stack_response.append(response)
response /= np.max(response) - np.min(response)
response -= np.min(response)
# response = 2*response - 1
responses.append(response)

responses = np.vstack(responses)
# responses /= np.max(responses) - np.min(responses)
# responses -= np.min(responses)
# responses = 2*responses - 1

In [ ]:
responses

In [ ]:
fig = plt.figure(figsize=(9, 4))
gs = fig.add_gridspec(1,3, width_ratios=[1,1,0.03],
                      wspace=0.1, hspace=0.1
                      )
cmap = 'Greys'
ax0 = fig.add_subplot(gs[0,0])
ax0.set_aspect('equal')
ax1 = fig.add_subplot(gs[0,1])
ax1.set_aspect('equal')
cax = fig.add_subplot(gs[0,2])
cax.set_aspect(10)

scatter0 = ax0.scatter([pos[0] for pos in sensor_pos], [pos[1] for pos in sensor_pos],
                       s=2e3, c=responses[0], cmap=plt.get_cmap(cmap), vmin=0, vmax=1,
                       edgecolors='black', linewidth=0.5)
scatter1 = ax1.scatter([pos[0] for pos in sensor_pos], [pos[1] for pos in sensor_pos],
                       s=2e3, c=responses[1], cmap=plt.get_cmap(cmap), vmin=0, vmax=1,
                       edgecolors='black', linewidth=0.5)
plt.colorbar(scatter0, cax=cax, ticks=np.arange(0,2,1))

for ax in fig.axes[:2]:
    ax.spines[:].set_visible(False)
    ax.set_xlim([-0.5, 3.5])
    ax.set_ylim([-0.5, 3.5])
    ax.set_xticks([])
    ax.set_yticks([])
cax.spines[:].set_visible(False)

ax0.set_title('Methane', fontsize=12)
ax1.set_title('Ethylene', fontsize=12)
plt.savefig('figs/resp_avg.pdf', bbox_inches='tight')
plt.savefig('figs/resp_avg.png', dpi=200, bbox_inches='tight')

In [ ]:
from scipy.optimize import curve_fit

data = stack_data[0]
times = stack_times[0]

def exp_func(x, a, b):
    return a * (1 - np.exp(-x/b))

# for i in range(16):
# Fit curve
params, covariance = curve_fit(exp_func, data[1, 10:125], times[10:125], p0=(1, 1))  # p0 is initial guess for [a, b]
a_fit, b_fit = params

In [ ]:
a_fit, b_fit

In [ ]:
plt.plot(data[1, 10:125])